In [1]:
from importlib import reload
import sys
sys.path.append('..\\..')
import numpy as np
from matplotlib import pyplot as plt
import Fluxonium_parameters as sqd
import QCreator.auxiliary_functions as auxfuncs
import QCreator.elements as elements
from QCreator import general_sample_creator as creator
creator

<module 'QCreator.general_sample_creator' from '/Users/guest1/Desktop/test/QCreator8/QCreator/QCreator/general_sample_creator.py'>

In [2]:
import Fluxonium_parameters as flux

In [3]:
reload(flux)

<module 'Fluxonium_parameters' from '/Users/guest1/Desktop/test/QCreator8/QCreator/Fluxonium_parameters.py'>

In [4]:
flux.sample.draw_design(True)
# flux.sample.draw_terminals()
# flux.sample.watch(True)


In [5]:
# flux.sample.write_to_gds()

## Это параметры трансмишен лайнов:

In [6]:

# это не используется:

tl_width = flux.tl_width
tl_gap = flux.tl_gap
tl_ground = flux.tl_ground

tl_width_qubit_to_meander = flux.tl_width_qubit_to_meander
tl_gap_qubit_to_meander = flux.tl_gap_qubit_to_meander 
tl_ground_qubit_to_meander = flux.tl_ground_qubit_to_meander

resonator_width = flux.resonator_width
resonator_gap = flux.resonator_gap
tl_res_ground = flux.tl_res_ground

meander_r = 50
grounding_width = 15

## Left qubit

In [7]:
qubits = {
    'q1':{
        'orientation': 'down',
        'coupler_position': np.asarray((1317, 4500/2 + 27)),
        'coupler_length': 550,
        'resonator_length': 6000,
        'qubit_id': 0,
    }, 
    'q2':{
        'orientation': 'down',
        'coupler_position': np.asarray((1300+760-15, 4500/2 + 27)),
        'coupler_length': 550,
        'resonator_length': 6000,
        'qubit_id': 0,
    }
}


In [8]:
# qubit, qubit_port, qubit_params, qubit_params2

# draw_qubit_with_resonator(qubit=flux.qubits[1], 
#                           qubit_port='left_coupler',  
#                           qubit_params=qubits['q1'], 
#                           qubit_params2=qubits['q2'])

qubit = flux.qubits[1]
qubit_port='left_coupler'
qubit_params=qubits['q1']
qubit_params2=qubits['q2']




w = [tl_width_qubit_to_meander, tl_res_ground, resonator_width]
s = [tl_gap_qubit_to_meander, tl_gap_qubit_to_meander, resonator_gap, resonator_gap]

coupler_position = qubit_params['coupler_position']
coupler_length = qubit_params['coupler_length']
orientation = qubit_params['orientation']

if orientation == 'down':
    w = w[::-1]
    s = s[::-1]
    
coupler = elements.CPWCoupler(
    'purcell-resonator coupler',
    [coupler_position, coupler_position + [coupler_length, 0]],
    w, s, tl_ground_qubit_to_meander, flux.sample.layer_configuration, r=meander_r,
                                    )
flux.sample.add(coupler)

if orientation=='up':
    fanout_left_grouping = [1, 3]
    fanout_left_grounding = [(2, 3)]
    fanout_left_port = 'up'
    fanout_right_grouping = [1, 3]
    fanout_right_grounding = [(0, 1)]
    fanout_right_port = 'down'
else:
    fanout_left_grouping = [0, 2]
    fanout_left_grounding = [(0, 1)]
    fanout_left_port = 'down'
    fanout_right_grouping = [0, 2]
    fanout_right_grounding = [(2, 3)]
    fanout_right_port = 'up'

fanout_left_1_left_qubit = flux.sample.fanout(o=coupler, port='port1', name='left end resonator fanout', grouping=fanout_left_grouping)
ground_left_1_left_qubit = flux.sample.ground(o=fanout_left_1_left_qubit, port='center', name='left end resonator grounding', grounding_width=grounding_width,
                   grounding_between=fanout_left_grounding)

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
flux.sample.connect_cpw(o1=ground_left_1_left_qubit, o2=flux.pads_left[0], port1='narrow', port2='port', name='feedline2', 
                       points=[])
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


fanout_right_1_left_qubit = flux.sample.fanout(o=coupler, port='port2', name='right end resonator fanout', grouping=fanout_right_grouping)
ground_right_1_left_qubit = flux.sample.ground(o=fanout_right_1_left_qubit, port='center', name='right end resonator grounding', grounding_width=grounding_width,
                   grounding_between=fanout_right_grounding)

# соединяем фанаут с нижними портами кубитов 
flux.sample.connect_cpw(o1=qubit,  
                        o2=fanout_right_1_left_qubit, 
                        port1=qubit_port,
                        port2=fanout_right_port, 
                        name='resonator-qubit cpw', 
                        points= [])
 

# add meander: 
closed_end_meander = flux.sample.connect_meander(name='closed end purcell', 
                                            o1=fanout_left_1_left_qubit, 
                                            port1='down',
                                            meander_length=2000,
                                            length_left=0,
                                            length_right=450,
                                            first_step_orientation='right',
                                            meander_orientation=np.pi,
                                            meander_type='round',
                                            airbridge=None,
                                            min_spacing=None, r=55,
                                              #end_point=end_point_closed_end, end_orientation=end_orientation_closed_end,
                                            bridge_part_decimation=1)

# соединим фанаут 
flux.sample.connect_cpw(o1=qubit,  
                        o2=fanout_right_1_left_qubit, 
                        port1=qubit_port,
                        port2=fanout_right_port, 
                        name='resonator-qubit cpw', 
                        points=[])


# print("ground_left.terminals", ground_left_1_left_qubit)

flux.sample.connect_cpw(o1=flux.fluxonium_right_up, o2=flux.pads_top[0], port1='qubit', port2='port', name='feedline2', 
                   points=[])



# ----------------------------------------------------------
# SECOND RESONATOR:

w = [tl_width_qubit_to_meander, tl_res_ground, resonator_width]
s = [tl_gap_qubit_to_meander, tl_gap_qubit_to_meander, resonator_gap, resonator_gap]

coupler_position = qubit_params2['coupler_position']
coupler_length = qubit_params2['coupler_length']
orientation = qubit_params2['orientation']

if orientation == 'down':
    w = w[::-1]
    s = s[::-1]
    
coupler = elements.CPWCoupler(
    'purcell-resonator coupler',
    [coupler_position, coupler_position + [coupler_length, 0]],
    w, s, tl_ground_qubit_to_meander, flux.sample.layer_configuration, r=meander_r,
                                    )
flux.sample.add(coupler)

if orientation=='up':
    fanout_left_grouping = [1, 3]
    fanout_left_grounding = [(2, 3)]
    fanout_left_port = 'up'
    fanout_right_grouping = [1, 3]
    fanout_right_grounding = [(0, 1)]
    fanout_right_port = 'down'
else:
    fanout_left_grouping = [0, 2]
    fanout_left_grounding = [(0, 1)]
    fanout_left_port = 'down'
    fanout_right_grouping = [0, 2]
    fanout_right_grounding = [(2, 3)]
    fanout_right_port = 'up'

fanout_left_2_left_qubit = flux.sample.fanout(o=coupler, port='port1', name='left end resonator fanout', grouping=fanout_left_grouping)
ground_left_2_left_qubit = flux.sample.ground(o=fanout_left_2_left_qubit, port='center', name='left end resonator grounding', grounding_width=grounding_width,
                   grounding_between=fanout_left_grounding)

fanout_right_2_left_qubit = flux.sample.fanout(o=coupler, port='port2', name='right end resonator fanout', grouping=fanout_right_grouping)
ground_right_2_left_qubit = flux.sample.ground(o=fanout_right_2_left_qubit, port='center', name='right end resonator grounding', grounding_width=grounding_width,
                   grounding_between=fanout_right_grounding)

flux.sample.connect_cpw(o1=qubit,  
                        o2=fanout_left_2_left_qubit, 
                        port1="right_coupler", 
                        port2=fanout_left_port, 
                        name='resonator-qubit cpw', 
                        points= [])
# add meander
closed_end_meander = flux.sample.connect_meander(name='closed end purcell', 
                        o1=fanout_right_2_left_qubit,  
                        port1='up',
                        meander_length=2000,
                        length_left=450,
                        length_right=0,
                        first_step_orientation='left',
                        meander_orientation=np.pi,
                        meander_type='round',
                        airbridge=None,
                        min_spacing=None, r=55,
                         #                end_point=end_point_closed_end, end_orientation=end_orientation_closed_end,
                        bridge_part_decimation=1)


## Херачим CPW 

In [9]:
# flux.sample.connect_cpw(o1=ground_left, o2=flux.pads_left[0], port1='narrow', port2='port', name='feedline2', 
#                        points=[])


## Right qubit 

In [10]:
qubits = {
    'q1':{
        'orientation': 'down',
        'coupler_position': np.asarray((2*flux.huns_offset + 1317, 4500/2 + 27)),
        'coupler_length': 550,
        'resonator_length': 6000,
        'qubit_id': 0,
    }, 
    'q2':{
        'orientation': 'down',
        'coupler_position': np.asarray((2*flux.huns_offset + 1300+760-15, 4500/2 + 27)),
        'coupler_length': 550,
        'resonator_length': 6000,
        'qubit_id': 0,
    }
}


In [11]:
qubit=flux.qubits[0]
qubit_port='left_coupler'
qubit_params=qubits['q1'] 
qubit_params2=qubits['q2']



w = [tl_width_qubit_to_meander, tl_res_ground, resonator_width]
s = [tl_gap_qubit_to_meander, tl_gap_qubit_to_meander, resonator_gap, resonator_gap]

coupler_position = qubit_params['coupler_position']
coupler_length = qubit_params['coupler_length']
orientation = qubit_params['orientation']

if orientation == 'down':
    w = w[::-1]
    s = s[::-1]
    
coupler = elements.CPWCoupler(
    'purcell-resonator coupler',
    [coupler_position, coupler_position + [coupler_length, 0]],
    w, s, tl_ground_qubit_to_meander, flux.sample.layer_configuration, r=meander_r,
                                    )
flux.sample.add(coupler)

if orientation=='up':
    fanout_left_grouping = [1, 3]
    fanout_left_grounding = [(2, 3)]
    fanout_left_port = 'up'
    fanout_right_grouping = [1, 3]
    fanout_right_grounding = [(0, 1)]
    fanout_right_port = 'down'
else:
    fanout_left_grouping = [0, 2]
    fanout_left_grounding = [(0, 1)]
    fanout_left_port = 'down'
    fanout_right_grouping = [0, 2]
    fanout_right_grounding = [(2, 3)]
    fanout_right_port = 'up'

# fanout_left_1_left_qubit

fanout_left_1_right_qubit = flux.sample.fanout(o=coupler, port='port1', name='left end resonator fanout', grouping=fanout_left_grouping)
ground_left_1_right_qubit = flux.sample.ground(o=fanout_left_1_right_qubit, port='center', name='left end resonator grounding', grounding_width=grounding_width,
                   grounding_between=fanout_left_grounding)

fanout_right_1_right_qubit = flux.sample.fanout(o=coupler, port='port2', name='right end resonator fanout', grouping=fanout_right_grouping)
ground_right_1_right_qubit = flux.sample.ground(o=fanout_right_1_right_qubit, port='center', name='right end resonator grounding', grounding_width=grounding_width,
                   grounding_between=fanout_right_grounding)


flux.sample.connect_cpw(o1=qubit,  
                        o2=fanout_right_1_right_qubit, 
                        port1=qubit_port,
                        port2=fanout_right_port, 
                        name='resonator-qubit cpw', 
                        points= [])
 

# add meander: 
closed_end_meander = flux.sample.connect_meander(name='closed end purcell', 
                                            o1=fanout_left_1_right_qubit, 
                                            port1='down',
                                            meander_length=2000,
                                            length_left=0,
                                            length_right=450,
                                            first_step_orientation='right',
                                            meander_orientation=np.pi,
                                            meander_type='round',
                                            airbridge=None,
                                            min_spacing=None, r=55,
                                            #  end_point=end_point_closed_end, end_orientation=end_orientation_closed_end,
                                            bridge_part_decimation=1)


flux.sample.connect_cpw(o1=qubit,  
                        o2=fanout_right_1_right_qubit, 
                        port1=qubit_port,
                        port2=fanout_right_port, 
                        name='resonator-qubit cpw', 
                        points=[])


# print("ground_left.terminals", ground_left.terminals)

flux.sample.connect_cpw(o1=flux.fluxonium_right_up, o2=flux.pads_top[0], port1='qubit', port2='port', name='feedline2', 
                   points=[])



# ------------------------------------------------
# SECOND RESONATOR:

w = [tl_width_qubit_to_meander, tl_res_ground, resonator_width]
s = [tl_gap_qubit_to_meander, tl_gap_qubit_to_meander, resonator_gap, resonator_gap]

coupler_position = qubit_params2['coupler_position']
coupler_length = qubit_params2['coupler_length']
orientation = qubit_params2['orientation']

if orientation == 'down':
    w = w[::-1]
    s = s[::-1]
    
coupler = elements.CPWCoupler(
    'purcell-resonator coupler',
    [coupler_position, coupler_position + [coupler_length, 0]],
    w, s, tl_ground_qubit_to_meander, flux.sample.layer_configuration, r=meander_r,
                                    )
flux.sample.add(coupler)

if orientation=='up':
    fanout_left_grouping = [1, 3]
    fanout_left_grounding = [(2, 3)]
    fanout_left_port = 'up'
    fanout_right_grouping = [1, 3]
    fanout_right_grounding = [(0, 1)]
    fanout_right_port = 'down'
else:
    fanout_left_grouping = [0, 2]
    fanout_left_grounding = [(0, 1)]
    fanout_left_port = 'down'
    fanout_right_grouping = [0, 2]
    fanout_right_grounding = [(2, 3)]
    fanout_right_port = 'up'

# fanout_left_2_left_qubit
fanout_left_2_right_qubit = flux.sample.fanout(o=coupler, port='port1', name='left end resonator fanout', grouping=fanout_left_grouping)
ground_left_2_right_qubit = flux.sample.ground(o=fanout_left_2_right_qubit, port='center', name='left end resonator grounding', grounding_width=grounding_width,
                   grounding_between=fanout_left_grounding)

fanout_right_2_right_qubit = flux.sample.fanout(o=coupler, port='port2', name='right end resonator fanout', grouping=fanout_right_grouping)
ground_right_2_right_qubit = flux.sample.ground(o=fanout_right_2_right_qubit, port='center', name='right end resonator grounding', grounding_width=grounding_width,
                   grounding_between=fanout_right_grounding)

flux.sample.connect_cpw(o1=qubit,  
                        o2=fanout_left_2_right_qubit, 
                        port1="right_coupler", 
                        port2=fanout_left_port, 
                        name='resonator-qubit cpw', 
                        points= [])
# add meander
closed_end_meander = flux.sample.connect_meander(name='closed end purcell', 
                        o1=fanout_right_2_right_qubit,  
                        port1='up',
                        meander_length=2000,
                        length_left=450,
                        length_right=0,
                        first_step_orientation='left',
                        meander_orientation=np.pi,
                        meander_type='round',
                        airbridge=None,
                        min_spacing=None, r=55,
                        #                 end_point=end_point_closed_end, end_orientation=end_orientation_closed_end,
                        bridge_part_decimation=1)


## Заземляем резонатор

In [12]:
flux.sample.ground(closed_end_meander[-1], 'port2', 'closed end meander grounding', 10, [[0, 2]])

RectGrounding closed end meander grounding

## Соединяем верх кубита и пад с помощью CPW

In [13]:
flux.sample.connect_cpw(o1=flux.fluxonium_left_up, o2=flux.pads_top[1], port1='qubit', port2='port', name='feedline2', 
                       points=[]) 

[CPW "feedline2", l=1037.000]

## Connect pads and ground with CPW 

In [14]:


# =>---==  ==       ==  ==---<=


flux.sample.connect_cpw(o1=ground_left_1_left_qubit, o2=flux.pads_left[0], port1='narrow', port2='port', name='feedline2', 
                        points=[])


flux.sample.connect_cpw(o1=ground_right_2_right_qubit, o2=flux.pads_right[0], port1='narrow', port2='port', name='feedline2', 
                       points=[])





[CPW "feedline2", l=725.056]

## Connect intermediate ground points 

In [15]:

# =>   ==  ==------==  ==   <=
# 

flux.sample.connect_cpw(o1=ground_right_2_left_qubit, o2=ground_left_1_right_qubit, port1='narrow', port2='narrow', name='feedline2', 
                        points=[])





[CPW "feedline2", l=2162.000]

In [16]:
flux.sample.connect_cpw(o1=ground_right_1_left_qubit, o2=ground_left_2_left_qubit, port1='narrow', port2='narrow', name='feedline2', 
                        points=[])
 
flux.sample.connect_cpw(o1=ground_right_1_right_qubit, o2=ground_left_2_right_qubit, port1='narrow', port2='narrow', name='feedline2', 
                        points=[])


# flux.sample.connect_cpw(o1=ground_right_2_left_qubit, o2=ground_left_1_right_qubit, port1='narrow', port2='narrow', name='feedline2', 
#                         points=[])



[CPW "feedline2", l=118.000]

### Overall

In [17]:
# flux.sample.connect_cpw(o1=ground_left, o2=flux.pads_left[0], port1='narrow', port2='port', name='feedline2', 
#                        points=[])


## Middle qubit 

In [18]:
# flux.sample.connect_cpw(o1=flux.fluxonium_down, o2=flux.pads_bottom[0], port1='qubit', port2='port', name='feedline2', 
#                        points=[]) 


# Draw full design 

In [19]:
flux.sample.draw_design(True)
#flux.sample.draw_terminals()
# flux.sample.watch(True)


/Users/guest1/opt/anaconda3/envs/test_env/lib/python3.7/site-packages/gdspy/path.py:617: UserWarning: [GDSPY] Possible inconsistency found in `FlexPath` due to sharp corner.
  "[GDSPY] Possible inconsistency found in "


In [20]:
flux.sample.write_to_gds()


Gds file has been writen here:  /Users/guest1/Desktop/test/QCreator8/QCreator/single qubits fluxonium.gds


# Calculate capacitance:

In [21]:
# watch the scheme

        #          1                  2
        #          |                  |
        #          |                  |
        #         ---                ---
        #         ---   ---||---     ---
        #          |    |       |     |
        #          |    |  jj   |     |
        # g----||----------|x|------------||---g
        #          l    |       |     r
        #               x       x
        #               x       x jj_chain
        #               x       x
        #               x       x
        #         qubit ---mmm--- inductance
        #               |       |
        #               /\      g
        #              |  |    
        #               ---      


In [22]:
flux.sample.draw_cap() 

In [23]:
flux.sample.watch(True)

TclError: can't invoke "winfo" command: application has been destroyed

In [24]:
cap_matrix = flux.sample.calculate_qubit_capacitance(flux.sample.qubit_cap_cells[1],
               flux.fluxonium_down,
               6)


Gds file has been writen here:  /Users/guest1/Desktop/test/QCreator8/QCreator/single qubits fluxonium.gds
For 0 conductor total length is:  7880
For 1 conductor total length is:  302
For 2 conductor total length is:  302
For 3 conductor total length is:  62
For 4 conductor total length is:  67
Sum : 8613
Data has been written into the file: /Users/guest1/Desktop/test/QCreator8/QCreator/mesh_4k_data
Time for fastcap job is:  1.208155870437622
Capacitance results have been writen here:  /Users/guest1/Desktop/test/QCreator8/QCreator/mesh_4k_results
Capacitance value in:  femtofarads
[<map object at 0x7fc9d7b1db90>, <map object at 0x7fc9d7b1dd90>, <map object at 0x7fc9d7b1d950>, <map object at 0x7fc9d7b0c190>, <map object at 0x7fc9d7b0c150>]
       0     1     2    3    4
0  108.7 -21.0 -21.0 -3.5 -3.5
1  -21.0  25.0  -0.1 -1.3 -0.0
2  -21.0  -0.1  25.0 -0.0 -1.3
3   -3.5  -1.3  -0.0  5.5 -0.0
4   -3.5  -0.0  -1.3 -0.0  5.5


In [25]:
cap_matrix = cap_matrix.iloc[1:,1:]

cap_matrix.columns = ['l', 'r', '1', '2']
cap_matrix.index = ['l', 'r', '1', '2']
cap_matrix

,l,r,1,2
l,25.0,-0.1,-1.3,-0.0
r,-0.1,25.0,-0.0,-1.3
1,-1.3,-0.0,5.5,-0.0
2,-0.0,-1.3,-0.0,5.5


In [26]:
flux.fluxonium_down.fill_capacitance_matrix(cap_matrix)

In [27]:
flux.fluxonium_down.capacitance_matrix

,l,r,1,2
l,25.0,-0.1,-1.3,-0.0
r,-0.1,25.0,-0.0,-1.3
1,-1.3,-0.0,5.5,-0.0
2,-0.0,-1.3,-0.0,5.5


In [28]:
flux.fluxonium_down.C

{'C_lg': 26.3, 'C_rg': 26.3, 'C_l1': 1.3, 'C_r2': 1.3, 'qubit': 0.5}

In [29]:
flux.fluxonium_down.capacitance_matrix['l']['r']

-0.1

Заполним capacitance_matrix у остальных 

In [30]:
flux.fluxonium_right_up.fill_capacitance_matrix(cap_matrix)
flux.fluxonium_left_up.fill_capacitance_matrix(cap_matrix)



In [31]:
flux.fluxonium_right_up.C 

{'C_lg': 26.3, 'C_rg': 26.3, 'C_l1': 1.3, 'C_r2': 1.3, 'qubit': 0.5}

---

# TLS / Simulations 

In [31]:
for object_ in flux.sample.objects:
    print(object_, ": ", object_.get_terminals())

<QCreator.elements.chip_edge_ground.ChipEdgeGround object at 0x7fcc474248d0> :  {}
<QCreator.elements.pad.Pad object at 0x7fcc47cbd710> :  {'port': Type: cpw, Position: (550, 2250.0), Orientation: 3.142
w: 4.0, s: 3, g: 21.0
reversed: True}
<QCreator.elements.pad.Pad object at 0x7fcc49310110> :  {'port': Type: cpw, Position: (6850.0, 2250.0), Orientation: 0
w: 4.0, s: 3, g: 21.0
reversed: True}
<QCreator.elements.pad.Pad object at 0x7fcc49491090> :  {'port': Type: cpw, Position: (1950.0, 3950.0), Orientation: 1.571
w: 4.0, s: 3, g: 21.0
reversed: True}
<QCreator.elements.pad.Pad object at 0x7fcc49491150> :  {'port': Type: cpw, Position: (5450.0, 3950.0), Orientation: 1.571
w: 4.0, s: 3, g: 21.0
reversed: True}
<QCreator.elements.pad.Pad object at 0x7fcc49491290> :  {'port': Type: cpw, Position: (3700.0, 550), Orientation: -1.571
w: 4.0, s: 3, g: 21.0
reversed: True}
<QCreator.elements.fluxonium.Fluxonium object at 0x7fcc49491310> :  {'qubit': Type: cpw, Position: (5450.0, 2913.0), Orie

In [32]:
circuit, connections, elements_ = flux.sample.get_tls(cutoff=2e11)
omegas, kappas, modes = circuit.get_modes()



In [33]:
kerr_matrix = circuit.get_perturbation([i for i in range(len(omegas))])
kerr_matrix


array([[-1.62020286e+00, -2.03846969e-02, -4.34993087e-02, ...,
        -1.61551719e-01, -7.12491265e+00, -4.62588965e+00],
       [-2.03846969e-02, -6.41178767e-05, -2.73644815e-04, ...,
        -1.01628719e-03, -4.48212963e-02, -2.91004790e-02],
       [-4.34993087e-02, -2.73644815e-04, -2.91968046e-04, ...,
        -2.16867537e-03, -9.56450522e-02, -6.20980886e-02],
       ...,
       [-1.61551719e-01, -1.01628719e-03, -2.16867537e-03, ...,
        -4.02711267e-03, -3.55215360e-01, -2.30625572e-01],
       [-7.12491265e+00, -4.48212963e-02, -9.56450522e-02, ...,
        -3.55215360e-01, -7.83302842e+00, -1.01712756e+01],
       [-4.62588965e+00, -2.91004790e-02, -6.20980886e-02, ...,
        -2.30625572e-01, -1.01712756e+01, -3.30187897e+00]])

In [35]:
modes_subsystems = circuit.define_modes_parameters(
    omegas / (2 * np.pi), modes, kerr_matrix,
    circuit.get_phi_dc(modes, circuit.i_dc()))

modes_subsystems 

{'0': {'subsystem_id': [8, 0],
  'Ec': array([[1.95191840e+02, 0.00000000e+00],
         [0.00000000e+00, 1.33038796e+12]]),
  'El': array([[7.79456194e-47, 0.00000000e+00],
         [0.00000000e+00, 1.14475024e-56]]),
  'Ej': [3.2910597847545334e-22,
   3.2910597847545334e-22,
   3.2910597847545334e-22,
   3.2910597847545334e-22,
   3.2910597847545334e-22,
   3.2910597847545334e-22],
  'alpha': array([[-2.59402050e-14, -2.89416436e-19],
         [-2.59402050e-14, -2.89416436e-19],
         [-2.59402050e-14, -2.89416436e-19]]),
  'dc_phase': array([[-0.06688787, -0.06688787],
         [-0.06688787, -0.06688787],
         [-0.06688787, -0.06688787]])},
 '1': {'subsystem_id': [8, 3],
  'Ec': array([[  195.19183956,     0.        ],
         [    0.        , 20415.32749875]]),
  'El': array([[7.79456194e-47, 0.00000000e+00],
         [0.00000000e+00, 7.45889996e-49]]),
  'Ej': [3.2910597847545334e-22,
   3.2910597847545334e-22,
   3.2910597847545334e-22,
   3.2910597847545334e-22,
   3.29

In [36]:
for i in range(102):
    circuit.plot_potential_1d(modes_subsystems[str(i)], phi=np.linspace(-2.5 * np.pi, 2.5 * np.pi, 301))



IndexError: index 3 is out of bounds for axis 0 with size 3

--------

----- 

------

# TRASH

# Add meanders

In [16]:
closed_end_meander = flux.sample.connect_meander(name='closed end purcell', 
                                                o1=flux.fluxonium_right_up, 
                                                port1='right_coupler',
                                                meander_length=3365,
                                                length_left=0,
                                                length_right=450,
                                                first_step_orientation='right',
                                                meander_orientation=np.pi/2,
                                                meander_type='round',
                                                airbridge=None,
                                                min_spacing=None, r=55,
#                                                 end_point=end_point_closed_end, end_orientation=end_orientation_closed_end,
                                                bridge_part_decimation=1)


                                                                     
    


SyntaxError: invalid syntax (545230636.py, line 16)

In [ ]:
closed_end_meander = flux.sample.connect_meander(name='closed end purcell', 
                                                o1=flux.fluxonium_right_up, 
                                                port1='right_coupler',
                                                meander_length=3365,
                                                length_left=0,
                                                length_right=450,
                                                first_step_orientation='right',
                                                meander_orientation=np.pi/2,
                                                meander_type='round',
                                                airbridge=None,
                                                min_spacing=None, r=55,
#                                                 end_point=end_point_closed_end, end_orientation=end_orientation_closed_end,
                                                bridge_part_decimation=1)




---

In [ ]:
# res_qubit_left_up,res_qubit_left_up_params = auxfuncs.draw_rounded_single_resonator_plus_qubit(flux.sample,
#                           coupler_start_x, coupler_start_y, coupler_length,
#                           flux.resonator_core,flux.resonator_gap, flux.resonator_ground,
#                           flux.tl_core, flux.tl_gap, flux.tl_ground, flux.resonator_tl_ground,
#                           closed_end_meander_length=2800, length_left=1500, length_right=740,
#                           min_bridge_spacing = None,
#                           airbridge = None, object1=flux.qubits[0], port='left_coupler',
#                           open_end_length = 1000,
#                           port_orientation='right',meander_orientation='left', direction_orientation='up',
#                           points_for_the_open_end=[(flux.qubits[0].terminals['left_coupler'].position[0],
#                                                     flux.qubits[0].terminals['left_coupler'].position[1]-200)])
# resonator_lengths.append(res_qubit_left_up_params[0])


In [15]:
# res_qubit_right_up,res_qubit_right_up_params = auxfuncs.draw_rounded_single_resonator_plus_qubit(flux.sample,
#                           5000, coupler_start_y, coupler_length,
#                           flux.resonator_core,flux.resonator_gap, flux.resonator_ground,
#                           flux.tl_core, flux.tl_gap, flux.tl_ground, flux.resonator_tl_ground,
#                           closed_end_meander_length=2000, length_left=1500, length_right=740,
#                           min_bridge_spacing = None,
#                           airbridge = None, object1=flux.qubits[1], port='left_coupler',
#                           open_end_length = 1000,
#                           port_orientation='right',meander_orientation='left', direction_orientation='up',
#                           points_for_the_open_end=[(flux.qubits[1].terminals['left_coupler'].position[0],
#                                                     flux.qubits[1].terminals['left_coupler'].position[1]-200)])
# resonator_lengths.append(res_qubit_right_up_params[0])


In [16]:
# res_qubit_right_up,res_qubit_right_up_params = auxfuncs.draw_rounded_single_resonator_plus_qubit(flux.sample,
#                           5000, coupler_start_y, coupler_length,
#                           flux.resonator_core,flux.resonator_gap, flux.resonator_ground,
#                           flux.tl_core, flux.tl_gap, flux.tl_ground, flux.resonator_tl_ground,
#                           closed_end_meander_length=1000, length_left=600, length_right=100,
#                           min_bridge_spacing = None,
#                           airbridge = None, object1=flux.qubits[1], port='left_coupler',
#                           open_end_length = 100,
#                           port_orientation='right', meander_orientation='left', direction_orientation='up',
#                           points_for_the_open_end=[(flux.qubits[1].terminals['left_coupler'].position[0],
#                                                     flux.qubits[1].terminals['left_coupler'].position[1]-200)])

# resonator_lengths.append(res_qubit_right_up_params[0])


In [17]:
# res_qubit_down,res_qubit_down_params = auxfuncs.draw_rounded_single_resonator_plus_qubit(flux.sample,
#                           3800, coupler_start_y, coupler_length,
#                           flux.resonator_core,flux.resonator_gap, flux.resonator_ground,
#                           flux.tl_core, flux.tl_gap, flux.tl_ground, flux.resonator_tl_ground,
#                           closed_end_meander_length=2000, length_left=1500, length_right=740,
#                           min_bridge_spacing = None,
#                           airbridge = None, object1=flux.qubits[2], port='left_coupler',
#                           open_end_length = 1000,
#                           port_orientation='right',meander_orientation='left', direction_orientation='down',
#                           points_for_the_open_end=[(flux.qubits[2].terminals['left_coupler'].position[0],
#                                                     flux.qubits[2].terminals['left_coupler'].position[1]+200)])
#
# resonator_lengths.append(res_qubit_down_params[0])


# Add test resonators

In [18]:


# res_coupler_t1,res_t1_params = auxfuncs.draw_rounded_single_resonator(sqd.sample,
#                           1500, coupler_start_y, 25 ,
#                           sqd.resonator_core,sqd.resonator_gap, sqd.resonator_ground,
#                           sqd.tl_core, sqd.tl_gap, sqd.tl_ground, 25,
#                           closed_end_meander_length=3400, length_left=300, length_right=300,
#                           min_bridge_spacing = None,
#                           airbridge = None,
#                           open_end_length = 150,
#                           port_orientation='left',meander_orientation='right', direction_orientation='up',transmission=True)

# resonator_lengths.append(res_t1_params[0])


In [19]:



res_coupler_t1_right ,res_t1_params_right = auxfuncs.draw_rounded_single_resonator(sqd.sample,
                          2500, coupler_start_y, 25 ,
                          sqd.resonator_core,sqd.resonator_gap, sqd.resonator_ground,
                          sqd.tl_core, sqd.tl_gap, sqd.tl_ground, 25,
                          closed_end_meander_length=3400, length_left=300, length_right=300,
                          min_bridge_spacing = None,
                          airbridge = None,
                          open_end_length = 150,
                          port_orientation='right',meander_orientation='right', direction_orientation='up',transmission=True)

resonator_lengths.append(res_t1_params_right[0])

res_coupler_t1_right.terminals

NameError: name 'coupler_start_y' is not defined

In [20]:
# res_coupler_t1,res_t1_params = auxfuncs.draw_rounded_single_resonator(sqd.sample,
#                           1500, coupler_start_y, 25+30+50,
#                           sqd.resonator_core,sqd.resonator_gap, sqd.resonator_ground,
#                           sqd.tl_core, sqd.tl_gap, sqd.tl_ground, 25,
#                           closed_end_meander_length=3900, length_left=300, length_right=300,
#                           min_bridge_spacing = None,
#                           airbridge = None,
#                           open_end_length = 150,
#                           port_orientation='right',meander_orientation='right', direction_orientation='down',transmission=True)

# resonator_lengths.append(res_t1_params[0])



In [21]:
# res_coupler_t_test,res_t_test_params = auxfuncs.draw_rounded_single_resonator(sqd.sample,
#                           2500, coupler_start_y, 25+30+50,
#                           sqd.resonator_core,sqd.resonator_gap, sqd.resonator_ground,
#                           sqd.tl_core, sqd.tl_gap, sqd.tl_ground, 25,
#                           closed_end_meander_length=4700, length_left=500, length_right=700,
#                           min_bridge_spacing = None,
#                           airbridge = None,
#                           open_end_length = 150,
#                           port_orientation='right',meander_orientation='right', direction_orientation='up',transmission=True)

# resonator_lengths.append(res_t_test_params[0])



In [22]:
# res_coupler_t2,res_t2_params = auxfuncs.draw_rounded_single_resonator(sqd.sample,
#                           3900, coupler_start_y, 25+30+50,
#                           sqd.resonator_core,sqd.resonator_gap, sqd.resonator_ground,
#                           sqd.tl_core, sqd.tl_gap, sqd.tl_ground, 25,
#                           closed_end_meander_length=3800, length_left=600, length_right=400,
#                           min_bridge_spacing = None,
#                           airbridge = None,
#                           open_end_length = 300,
#                           port_orientation='left',meander_orientation='left', direction_orientation='down',transmission=True)

# resonator_lengths.append(res_t2_params[0])


# Meander


In [23]:
flux.fluxonium_right_up.terminals

{'qubit': Type: cpw, Position: (1955, 2913.0), Orientation: 4.712
 w: 10, s: 6, g: 6
 reversed: True,
 'left_coupler': Type: cpw, Position: (1875.0, 2872.0), Orientation: 1.571
 w: 21, s: 12, g: 6
 reversed: True,
 'right_coupler': Type: cpw, Position: (2035.0, 2872.0), Orientation: 1.571
 w: 21, s: 12, g: 6
 reversed: True}

In [24]:

# !!!!!!!!!!!!!!!!

closed_end_meander = flux.sample.connect_meander(name='closed end purcell', 
                                                o1=flux.fluxonium_right_up, 
                                                port1='left_coupler',
                                                meander_length=3365,
                                                length_left=450,
                                                length_right=-100,
                                                first_step_orientation='left',
                                                meander_orientation=-np.pi/2,
                                                meander_type='round',
                                                airbridge=None,
                                                min_spacing=None, r=55,
#                                                 end_point=end_point_closed_end, end_orientation=end_orientation_closed_end,
                                                bridge_part_decimation=1)



In [25]:
closed_end_meander = flux.sample.connect_meander(name='closed end purcell', 
                                                o1=flux.fluxonium_right_up, 
                                                port1='right_coupler',
                                                meander_length=3365,
                                                length_left=0,
                                                length_right=450,
                                                first_step_orientation='right',
                                                meander_orientation=np.pi/2,
                                                meander_type='round',
                                                airbridge=None,
                                                min_spacing=None, r=55,
#                                                 end_point=end_point_closed_end, end_orientation=end_orientation_closed_end,
                                                bridge_part_decimation=1)




In [26]:
# coupler2_w = [flux.resonator_core, 2*flux.resonator_ground, flux.resonator_core]
# coupler2_s = [flux.resonator_gap, flux.resonator_gap, flux.resonator_gap, flux.resonator_gap]

 

# main_coupler2 = elements.CPWCoupler('purcell-resonator coupler',
#                                        [(1000, 1000),
#                                         (1000 + 1000,
#                                          1500)],
#                                        coupler2_w, coupler2_s, flux.resonator_ground, flux.layers_configuration, r=55,
#                                         )
# flux.sample.add(main_coupler2)
# main_coupler2.terminals 

In [27]:

# coupler_start_x3 = 3370
# coupler_start_y_down = 2342.0





# purcell3 = auxfuncs.draw_purcell(flux.sample,
#                           coupler_start_x3, coupler_start_y_down, 400,
#                           flux.resonator_core,flux.resonator_gap, flux.resonator_ground,
#                           flux.tl_core, flux.tl_gap, 0,10, first_step_orientation='right',
#                         closed_end_meander_length=3365, length_left=450, length_right=350,
#                             open_end_length_left=0, open_end_length_right=0,open_end_length = 0,
#                                  open_end_first_step_orientation='left',
#                           port_orientation='right', direction_orientation='down', meander_r=55, airbridge=None, 
#                              min_bridge_spacing=None, coupler2_length=120,
#                                  closed_end_res_meander_length = 3403, res_length_left = 778, res_length_right = 40, 
#                                  object1=flux.fluxonium_left_up, port='right_coupler', bridge_part_decimation=2, open_end_angle = -np.pi/2)





# Connect resonators and make a feedline

In [28]:
flux.fluxonium_down.terminals 

{'qubit': Type: cpw, Position: (4000, 1313.0), Orientation: 1.571
 w: 10, s: 6, g: 6
 reversed: True,
 'left_coupler': Type: cpw, Position: (4080.0, 1328.0), Orientation: 4.712
 w: 21, s: 12, g: 6
 reversed: True,
 'right_coupler': None}

In [29]:
flux.sample.connect_cpw(o1=flux.p1, o2=flux.p2, port1='port', port2='port', name='feedline1', 
                       points=[])

# flux.sample.connect_cpw(o1=flux.fluxonium_down, o2=flux.pads_bottom[0], port1='qubit', port2='port', name='feedline2', 
#                        points=[])

flux.sample.connect_cpw(o1=flux.fluxonium_right_up, o2=flux.pads_top[0], port1='qubit', port2='port', name='feedline2', 
                       points=[])






10.0 10.0
6.0 6.0
6.0 6.0
10 10.0
6 6.0
6 6.0


[CPW "feedline2", l=1037.012]

In [30]:
flux.pads_top[0].terminal 

Type: cpw, Position: (1950.0, 3950.0), Orientation: 1.571
w: 10.0, s: 6.0, g: 6.0
reversed: True

In [31]:
flux.fluxonium_down.terminals

{'qubit': Type: cpw, Position: (4000, 1313.0), Orientation: 1.571
 w: 10, s: 6, g: 6
 reversed: True,
 'left_coupler': Type: cpw, Position: (4080.0, 1328.0), Orientation: 4.712
 w: 21, s: 12, g: 6
 reversed: True,
 'right_coupler': None}

In [32]:
flux.pads_bottom[0].terminal 

Type: cpw, Position: (3700.0, 550), Orientation: -1.571
w: 10.0, s: 6.0, g: 6.0
reversed: True

In [33]:
flux.pads_bottom[0] 


In [34]:
flux.sample.draw_design(True)
flux.sample.draw_terminals()
flux.sample.watch(True)

# Calculate capacitances

In [35]:
flux.sample.draw_cap() 

In [36]:
# смотреть параметр calculate_capatitance 
caps=flux.sample.calculate_qubit_capacitance(flux.sample.qubit_cap_cells[0],
               flux.qubits[0],
               5)

IndexError: list index out of range

In [37]:
caps

NameError: name 'caps' is not defined

In [38]:
flux.sample.watch(True)